In [1]:
!pip install ktrain


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 91.6 MB/s 
     |████████████████████████████████| 981 kB 66.1 MB/s 
     |████████████████████████████████| 265 kB 55.3 MB/s 
     |████████████████████████████████| 3.8 MB 71.1 MB/s 
     |████████████████████████████████| 1.3 MB 64.6 MB/s 
     |████████████████████████████████| 468 kB 67.1 MB/s 
     |████████████████████████████████| 880 kB 68.2 MB/s 
     |████████████████████████████████| 182 kB 74.3 MB/s 
     |████████████████████████████████| 7.6 MB 68.4 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.10-py3-none-any.whl size=25312982 sha256=076191b3592e7c74b266eb041f140ed78e22c3a604581d0fe7dd53967c34a34f
  Stored in directory: /root/.cache/pip/wheels/03/2c/bd/18821c5ecd2a95cc5d6fc61bb08db2f70dd2c952b46fbe8d7b
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33516 sha256=5e201841c749ca9494a831

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import ktrain
from ktrain import text

In [3]:
from platform import python_version

print(python_version())

3.8.15


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
## Start from this Cell
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
dataframe=pd.read_csv('/content/drive/MyDrive/kaggle disaster Tweet/clean_train.csv')

In [7]:
dataframe=dataframe[dataframe['text'].notnull()]

In [9]:
dataframe['text'].isnull().sum()

0

In [8]:
dataframe.head()

,text,label
0,ded reason may alah forgive u al,1
1,forest fire near la ronge sask canada,1
2,al resident asked shelter place notified ofice...,1
3,people receive evacuation order california,1
4,got sent photo ruby smoke pours schol,1


In [10]:
train = dataframe.iloc[0:,:]
valid=dataframe.iloc[7200:,:]
# test=dataframe.iloc[7100:,:]

In [11]:
X_train=train['text'].values
X_val=valid['text'].values
y_train=train['label'].values
y_val=valid['label'].values


In [12]:
MODEL_NAME = 'dbmdz/bert-base-italian-xxl-cased'
#MODEL_NAME = 'xlm-roberta-base'

t = text.Transformer(MODEL_NAME, maxlen=400, classes=np.array([0,1]))
trn = t.preprocess_train(np.array(X_train),np.array(y_train))
val = t.preprocess_test(np.array(X_val), np.array(y_val))
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=8)

/usr/local/lib/python3.8/dist-packages/ktrain/text/preprocessor.py:489: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 9
	95percentile : 14
	99percentile : 17


Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 8
	95percentile : 15
	99percentile : 17


Downloading:   0%|          | 0.00/520M [00:00<?, ?B/s]

In [ ]:
learner.fit_onecycle(1e-5, 10)




begin training using onecycle policy with max lr of 1e-05...
Epoch 1/10
906/906 [==============================] - 627s 691ms/step - loss: 0.3182 - accuracy: 0.8766 - val_loss: 0.0330 - val_accuracy: 1.0000
Epoch 2/10
906/906 [==============================] - 628s 691ms/step - loss: 0.3053 - accuracy: 0.8798 - val_loss: 0.0243 - val_accuracy: 1.0000
Epoch 3/10
906/906 [==============================] - 627s 691ms/step - loss: 0.2729 - accuracy: 0.8914 - val_loss: 0.0353 - val_accuracy: 1.0000
Epoch 4/10
577/906 [==================>...........] - ETA: 3:47 - loss: 0.2371 - accuracy: 0.9107

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/kaggle disaster Tweet/test.csv')

In [ ]:
test.head()

In [ ]:
test.drop(['id','keyword','location'],axis=1,inplace=True)

In [ ]:
test.head()

In [ ]:
pred = ktrain.get_predictor(learner.model,preproc = t)

In [ ]:
predictor=pred.predict(X_test, return_proba=True)

In [ ]:
predictor_1=pred.predict(X_test)

In [ ]:
predictor

array([[0.01310051, 0.9868995 ],
       [0.49233   , 0.50767   ],
       [0.01516138, 0.9848386 ],
       ...,
       [0.01145592, 0.98854405],
       [0.18665344, 0.8133465 ],
       [0.9963186 , 0.00368136]], dtype=float32)

In [ ]:
y_predicted = predictor.flatten()


In [ ]:
y_predicted

array([0.01310051, 0.9868995 , 0.49233   , ..., 0.8133465 , 0.9963186 ,
       0.00368136], dtype=float32)

In [ ]:
y_predicted=y_predicted.tolist()

In [ ]:
len(y_predicted)

1726

In [ ]:
res = [i for j, i in enumerate(y_predicted) if j % 2 != 0]

In [ ]:
res

[0.9868994951248169,
 0.507669985294342,
 0.984838604927063,
 0.00368135841563344,
 0.0063521298579871655,
 0.9838348627090454,
 0.9331104755401611,
 0.9845689535140991,
 0.00368135841563344,
 0.007868430577218533,
 0.9727854132652283,
 0.9686563014984131,
 0.9907686114311218,
 0.00368135841563344,
 0.9890083074569702,
 0.016093069687485695,
 0.00368135841563344,
 0.004334272816777229,
 0.007032795809209347,
 0.5160664916038513,
 0.9869468212127686,
 0.024388065561652184,
 0.9846954941749573,
 0.9902046918869019,
 0.004334272816777229,
 0.9846954941749573,
 0.004334272816777229,
 0.9882237911224365,
 0.00368135841563344,
 0.9883983731269836,
 0.8632301092147827,
 0.009192495606839657,
 0.007092955056577921,
 0.9864181876182556,
 0.00368135841563344,
 0.00368135841563344,
 0.006395340897142887,
 0.007461024913936853,
 0.019433826208114624,
 0.00368135841563344,
 0.07994653284549713,
 0.00368135841563344,
 0.9885911345481873,
 0.014240976423025131,
 0.004334272816777229,
 0.9830850958824

In [ ]:
predictions_poss = res


In [ ]:
predictions_poss

[0.9868994951248169,
 0.507669985294342,
 0.984838604927063,
 0.00368135841563344,
 0.0063521298579871655,
 0.9838348627090454,
 0.9331104755401611,
 0.9845689535140991,
 0.00368135841563344,
 0.007868430577218533,
 0.9727854132652283,
 0.9686563014984131,
 0.9907686114311218,
 0.00368135841563344,
 0.9890083074569702,
 0.016093069687485695,
 0.00368135841563344,
 0.004334272816777229,
 0.007032795809209347,
 0.5160664916038513,
 0.9869468212127686,
 0.024388065561652184,
 0.9846954941749573,
 0.9902046918869019,
 0.004334272816777229,
 0.9846954941749573,
 0.004334272816777229,
 0.9882237911224365,
 0.00368135841563344,
 0.9883983731269836,
 0.8632301092147827,
 0.009192495606839657,
 0.007092955056577921,
 0.9864181876182556,
 0.00368135841563344,
 0.00368135841563344,
 0.006395340897142887,
 0.007461024913936853,
 0.019433826208114624,
 0.00368135841563344,
 0.07994653284549713,
 0.00368135841563344,
 0.9885911345481873,
 0.014240976423025131,
 0.004334272816777229,
 0.9830850958824

In [ ]:
df.head()

,albert_pos,albert_neg,albert_predictions,bert_pos,bert_neg,bert_predictions,distill_pos,distill_neg,distill_predictions,roberta_pos,roberta_neg,roberta_predictions
0,0.939936,0.060064,1,0.983669,0.016331,1,0.984251,0.015749,1,0.989997,0.010003,1
1,0.077037,0.922963,0,0.455628,0.544372,0,0.060348,0.939652,0,0.704662,0.295338,1
2,0.059596,0.940404,0,0.966730,0.033270,1,0.949289,0.050711,1,0.984212,0.015788,1
3,0.012473,0.987527,0,0.003419,0.996581,0,0.004605,0.995395,0,0.001929,0.998071,0
4,0.025207,0.974793,0,0.008775,0.991225,0,0.008523,0.991477,0,0.005626,0.994374,0


In [ ]:
len(predictions_poss)

863

In [ ]:
df['clean_bert_pos']=predictions_poss

In [ ]:
df['clean_bert_pos']

0      0.986899
1      0.507670
2      0.984839
3      0.003681
4      0.006352
         ...   
858    0.004334
859    0.986588
860    0.988544
861    0.813347
862    0.003681
Name: clean_bert_pos, Length: 863, dtype: float64

In [ ]:
res[:] = [1 - number for number in res]

In [ ]:
res

[0.013100504875183105,
 0.49233001470565796,
 0.015161395072937012,
 0.9963186415843666,
 0.9936478701420128,
 0.01616513729095459,
 0.06688952445983887,
 0.015431046485900879,
 0.9963186415843666,
 0.9921315694227815,
 0.02721458673477173,
 0.031343698501586914,
 0.009231388568878174,
 0.9963186415843666,
 0.010991692543029785,
 0.9839069303125143,
 0.9963186415843666,
 0.9956657271832228,
 0.9929672041907907,
 0.4839335083961487,
 0.013053178787231445,
 0.9756119344383478,
 0.015304505825042725,
 0.009795308113098145,
 0.9956657271832228,
 0.015304505825042725,
 0.9956657271832228,
 0.011776208877563477,
 0.9963186415843666,
 0.011601626873016357,
 0.13676989078521729,
 0.9908075043931603,
 0.9929070449434221,
 0.013581812381744385,
 0.9963186415843666,
 0.9963186415843666,
 0.9936046591028571,
 0.9925389750860631,
 0.9805661737918854,
 0.9963186415843666,
 0.9200534671545029,
 0.9963186415843666,
 0.011408865451812744,
 0.9857590235769749,
 0.9956657271832228,
 0.01691490411758423,


In [ ]:
predictions_neg=res

In [ ]:
predictions_neg

[0.013100504875183105,
 0.49233001470565796,
 0.015161395072937012,
 0.9963186415843666,
 0.9936478701420128,
 0.01616513729095459,
 0.06688952445983887,
 0.015431046485900879,
 0.9963186415843666,
 0.9921315694227815,
 0.02721458673477173,
 0.031343698501586914,
 0.009231388568878174,
 0.9963186415843666,
 0.010991692543029785,
 0.9839069303125143,
 0.9963186415843666,
 0.9956657271832228,
 0.9929672041907907,
 0.4839335083961487,
 0.013053178787231445,
 0.9756119344383478,
 0.015304505825042725,
 0.009795308113098145,
 0.9956657271832228,
 0.015304505825042725,
 0.9956657271832228,
 0.011776208877563477,
 0.9963186415843666,
 0.011601626873016357,
 0.13676989078521729,
 0.9908075043931603,
 0.9929070449434221,
 0.013581812381744385,
 0.9963186415843666,
 0.9963186415843666,
 0.9936046591028571,
 0.9925389750860631,
 0.9805661737918854,
 0.9963186415843666,
 0.9200534671545029,
 0.9963186415843666,
 0.011408865451812744,
 0.9857590235769749,
 0.9956657271832228,
 0.01691490411758423,


In [ ]:
df['clean_bert_neg']=predictions_neg

In [ ]:
predictor_1

[1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,


In [ ]:
df['clean_bert_predictions']=predictor_1

In [ ]:
df.to_csv('/content/drive/MyDrive/MediaEval/DisasterMM/data/Results.csv',index=False)


In [ ]:
df.head()

,albert_pos,albert_neg,albert_predictions,bert_pos,bert_neg,bert_predictions,distill_pos,distill_neg,distill_predictions,roberta_pos,roberta_neg,roberta_predictions,clean_bert_pos,clean_bert_neg,clean_bert_predictions
0,0.939936,0.060064,1,0.983669,0.016331,1,0.984251,0.015749,1,0.989997,0.010003,1,0.986899,0.013101,1
1,0.077037,0.922963,0,0.455628,0.544372,0,0.060348,0.939652,0,0.704662,0.295338,1,0.507670,0.492330,1
2,0.059596,0.940404,0,0.966730,0.033270,1,0.949289,0.050711,1,0.984212,0.015788,1,0.984839,0.015161,1
3,0.012473,0.987527,0,0.003419,0.996581,0,0.004605,0.995395,0,0.001929,0.998071,0,0.003681,0.996319,0
4,0.025207,0.974793,0,0.008775,0.991225,0,0.008523,0.991477,0,0.005626,0.994374,0,0.006352,0.993648,0


In [ ]:
unique_elements, counts_elements = np.unique(y_predicted, return_counts=True)

In [ ]:
unique_elements

array([0.00368136, 0.00433427, 0.00433428, ..., 0.99529761, 0.99566573,
       0.99631858])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [ ]:
print(confusion_matrix(y_test,predictor_1))

[[485  29]
 [ 21 328]]


In [ ]:
print(classification_report(y_test,predictor_1))
print(accuracy_score(y_test, predictor_1))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       514
           1       0.92      0.94      0.93       349

    accuracy                           0.94       863
   macro avg       0.94      0.94      0.94       863
weighted avg       0.94      0.94      0.94       863

0.9420625724217845


In [ ]:
predictor

array([[0.01310051, 0.9868995 ],
       [0.49233   , 0.50767   ],
       [0.01516138, 0.9848386 ],
       ...,
       [0.01145592, 0.98854405],
       [0.18665344, 0.8133465 ],
       [0.9963186 , 0.00368136]], dtype=float32)